In [1]:
from bert import BertModel
import datasets
# Load the dataset
imdb_data = datasets.load_from_disk("data/5000_sampled_imdb")

/home/lynie/miniconda3/envs/eff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import random
random.seed(42)
imdb_data = imdb_data.shuffle()
train_data = imdb_data[:800]
test_data = imdb_data[800:]

In [3]:
from utils import *
import numpy as np

bert_config = ModelArguments()
bert_config.num_labels = 3
bert_config.model = "bert-base-uncased"
bert_config.cache_size = 32
bert_config.batch_size = 8
bert_config.num_epochs = 5
bert_config.confidence = 4
bert_config.cost = 20
# bert_config.class_weight = "balanced"

bert_model = BertModel(bert_config)

for x in range(len(train_data['text'])):
    bert_model.cache_add(train_data['text'][x], train_data['label'][x])
    if len(bert_model.online_cache['label']) == bert_config.cache_size:
        online_data = GenericDataset(bert_model.online_cache)
        online_dataloader = DataLoader(online_data, batch_size=bert_config.batch_size, shuffle=True)
        bert_model.train_online(online_dataloader)
        bert_model.cache_clear()

correct = 0
total_conf, correct_conf, wrong_conf = 0, 0, 0
for x in range(len(test_data['text'])):
    pred, conf = bert_model.predict(test_data['text'][x])
    total_conf += conf
    if pred == test_data['label'][x]:
        correct += 1
#         correct_conf += conf
#     else:
#         wrong_conf += conf
# print("Accuracy:", correct/len(test_data['text']))
# print("Average confidence:", total_conf/len(test_data['text']))
# print("Average confidence for correct predictions:", correct_conf/correct)
# print("Average confidence for wrong predictions:", wrong_conf/(len(test_data['text'])-correct))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT Model loaded


Epoch 4 Loss: 2.9753: 100%|██████████| 4/4 [00:00<00:00,  4.31it/s]


Accuracy: 0.475
Average confidence: 0.999998049698132
Average confidence for correct predictions: 0.999999723846584
Average confidence for wrong predictions: 0.9999965349923894
